In [ ]:
!pip install pandas as pd

ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as


In [ ]:
!pip install scikit-learn

In [ ]:
!pip install numpy as np

ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as


In [ ]:
# training RANDOMFOREST on dataset and print the head of the file for a review

In [ ]:
import pandas as pd
import numpy as np

# Load the dataset
file_path = '/content/crop yield data sheet.xlsx'
data = pd.read_excel(file_path)

print(data.head())

features = ['Rain Fall (mm)', 'Fertilizer', 'Temperatue', 'Nitrogen (N)', 'Phosphorus (P)', 'Potassium (K)']
for feature in features:
    print(f"Checking column: {feature}")
    non_numeric = data[pd.to_numeric(data[feature], errors='coerce').isna()]
    if not non_numeric.empty:
        print(f"Non-numeric values found in column {feature}:")
        print(non_numeric)

data[features] = data[features].apply(pd.to_numeric, errors='coerce')
data = data.dropna()

print(data.head())


   Rain Fall (mm)  Fertilizer Temperatue  Nitrogen (N)  Phosphorus (P)  \
0          1230.0        80.0         28          80.0            24.0   
1           480.0        60.0         36          70.0            20.0   
2          1250.0        75.0         29          78.0            22.0   
3           450.0        65.0         35          70.0            19.0   
4          1200.0        80.0         27          79.0            22.0   

   Potassium (K)  Yeild (Q/acre)  
0           20.0            12.0  
1           18.0             8.0  
2           19.0            11.0  
3           18.0             9.0  
4           19.0            11.0  
Checking column: Rain Fall (mm)
Non-numeric values found in column Rain Fall (mm):
     Rain Fall (mm)  Fertilizer Temperatue  Nitrogen (N)  Phosphorus (P)  \
99              NaN         NaN        NaN           NaN             NaN   
100             NaN         NaN        NaN           NaN             NaN   
101             NaN         NaN   

In [ ]:
!pip install openpyxl

In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
#the model is RANDOM FOREST REGRESSOR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

file_path = '/content/crop yield data sheet.xlsx'
data = pd.read_excel(file_path)

print("Initial Data Preview:")       #displays the head of each column
print(data.head())

features = ['Rain Fall (mm)', 'Fertilizer', 'Temperatue', 'Nitrogen (N)', 'Phosphorus (P)', 'Potassium (K)']         #checking for non-numeric values in the feature columns
for feature in features:
    print(f"Checking column: {feature}")
    non_numeric = data[pd.to_numeric(data[feature], errors='coerce').isna()]
    if not non_numeric.empty:
        print(f"Non-numeric values found in column {feature}:")
        print(non_numeric)


data[features] = data[features].apply(pd.to_numeric, errors='coerce')    #replace non-numeric values with NaN and then fill with the column mean or drop them
data = data.dropna()

print("Cleaned Data Preview:")           #display cleaned data
print(data.head())

X = data[features]                      #feature columns

y = data['Yeild (Q/acre)']              #target column

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)    #train test split

model = RandomForestRegressor(n_estimators=100, random_state=42)        #initialization

model.fit(X_train, y_train)            #training model

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)            #model evaluation
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.2f}")

def predict_crop_yield(rain_fall, fertilizer, temperature, nitrogen, phosphorus, potassium):             #real time predictor
    input_data = pd.DataFrame([[rain_fall, fertilizer, temperature, nitrogen, phosphorus, potassium]], columns=features)  #create a df

    predicted_yield = model.predict(input_data)    #predict yield

    return predicted_yield[0]
def get_user_input():
    try:
        rain_fall = float(input("Enter Rain Fall (mm): "))
        fertilizer = float(input("Enter Fertilizer amount: "))
        temperature = float(input("Enter Temperature: "))              #user input
        nitrogen = float(input("Enter Nitrogen (N) amount: "))
        phosphorus = float(input("Enter Phosphorus (P) amount: "))
        potassium = float(input("Enter Potassium (K) amount: "))

        predicted_yield = predict_crop_yield(rain_fall, fertilizer, temperature, nitrogen, phosphorus, potassium)
        print(f"Predicted Crop Yield: {predicted_yield:.2f}")

        provide_suggestions(predicted_yield, rain_fall, fertilizer, temperature, nitrogen, phosphorus, potassium)   #providing suggestions based on the input vectors

    except ValueError:
        print("Invalid input. Please enter numeric values.")

def provide_suggestions(predicted_yield, rain_fall, fertilizer, temperature, nitrogen, phosphorus, potassium):
    # Example threshold values (these should be adjusted based on domain knowledge)
    high_yield_threshold = 1000.0
    low_fertilizer_threshold = 50.0
    high_rain_fall_threshold = 1200.0
    optimal_temperature_range = (20, 30)  # Optimal temperature range in Celsius
    low_nitrogen_threshold = 30.0
    high_nitrogen_threshold = 80.0
    low_phosphorus_threshold = 15.0
    high_phosphorus_threshold = 30.0
    low_potassium_threshold = 10.0
    high_potassium_threshold = 25.0

    print("\nSuggestions:")

    def print_suggestion(value, ideal_range, name):
        if isinstance(ideal_range, tuple):
            if ideal_range[0] <= value <= ideal_range[1]:
                print(f"{name}: Current = {value}, Ideal = {ideal_range[0]} - {ideal_range[1]}, Within ideal range")
                return 0
            else:
                if value < ideal_range[0]:
                    diff = ideal_range[0] - value
                    print(f"{name}: Current = {value}, Ideal = {ideal_range[0]} - {ideal_range[1]}, Increase by {diff}")
                    return diff
                else:
                    diff = value - ideal_range[1]
                    print(f"{name}: Current = {value}, Ideal = {ideal_range[0]} - {ideal_range[1]}, Decrease by {diff}")
                    return diff
        else:
            if value < ideal_range:
                diff = ideal_range - value
                print(f"{name}: Current = {value}, Ideal > {ideal_range}, Increase by {diff}")
                return diff
            else:
                print(f"{name}: Current = {value}, Ideal > {ideal_range}, Within ideal range")
                return 0

    diff_rainfall = print_suggestion(rain_fall, high_rain_fall_threshold, "Rain Fall (mm)")
    diff_fertilizer = print_suggestion(fertilizer, low_fertilizer_threshold, "Fertilizer")
    diff_temperature = print_suggestion(temperature, optimal_temperature_range, "Temperature (°C)")
    diff_nitrogen = print_suggestion(nitrogen, (low_nitrogen_threshold, high_nitrogen_threshold), "Nitrogen (N)")
    diff_phosphorus = print_suggestion(phosphorus, (low_phosphorus_threshold, high_phosphorus_threshold), "Phosphorus (P)")
    diff_potassium = print_suggestion(potassium, (low_potassium_threshold, high_potassium_threshold), "Potassium (K)")

    if predicted_yield > high_yield_threshold:
        print("The predicted yield is high. Consider reducing fertilizer use to optimize costs.")
    else:
        if fertilizer < low_fertilizer_threshold:
            print("The predicted yield is low. Consider increasing fertilizer application.")
        else:
            print("Check other factors that may affect yield such as pest control or soil health")        #suggestions

    if rain_fall > high_rain_fall_threshold:
        print("Heavy rainfall detected. Ensure proper drainage to prevent waterlogging.")

    if not (optimal_temperature_range[0] <= temperature <= optimal_temperature_range[1]):
        print(f"Temperature is out of the optimal range ({optimal_temperature_range[0]}-{optimal_temperature_range[1]}°C). Consider measures to regulate field temperature.")

    if nitrogen < low_nitrogen_threshold:
        print("Nitrogen levels are low. Consider increasing nitrogen-based fertilizers.")
    elif nitrogen > high_nitrogen_threshold:
        print("Nitrogen levels are high. Avoid over-fertilization to prevent negative environmental impacts.")

    if phosphorus < low_phosphorus_threshold:
        print("Phosphorus levels are low. Consider increasing phosphorus-based fertilizers.")
    elif phosphorus > high_phosphorus_threshold:
        print("Phosphorus levels are high. Avoid over-fertilization to prevent nutrient runoff.")

    if potassium < low_potassium_threshold:
        print("Potassium levels are low. Consider increasing potassium-based fertilizers.")
    elif potassium > high_potassium_threshold:
        print("Potassium levels are high. Avoid over-fertilization to prevent soil imbalance.")

get_user_input()


Initial Data Preview:
   Rain Fall (mm)  Fertilizer Temperatue  Nitrogen (N)  Phosphorus (P)  \
0          1230.0        80.0         28          80.0            24.0   
1           480.0        60.0         36          70.0            20.0   
2          1250.0        75.0         29          78.0            22.0   
3           450.0        65.0         35          70.0            19.0   
4          1200.0        80.0         27          79.0            22.0   

   Potassium (K)  Yeild (Q/acre)  
0           20.0            12.0  
1           18.0             8.0  
2           19.0            11.0  
3           18.0             9.0  
4           19.0            11.0  
Checking column: Rain Fall (mm)
Non-numeric values found in column Rain Fall (mm):
     Rain Fall (mm)  Fertilizer Temperatue  Nitrogen (N)  Phosphorus (P)  \
99              NaN         NaN        NaN           NaN             NaN   
100             NaN         NaN        NaN           NaN             NaN   
101         

In [ ]:
#uses of performance metrics like MEAN SQUARED ERROR(MSE) AND R-SQUARED VALUE()
#MSE: MSE is calculated as the average of the squared differences between predicted values and actual values.
#MSE value 0.51 indicates that, on average, the squared differences between the actual crop yields and the predicted yields are 0.51.
#R-SQUARED VALUE:The R-squared (R²) value is a measure of how well the predicted values match the actual values. It represents the proportion of the variance in the dependent variable that is predictable from the independent variables.
#R-SQUARED VALUE of 0.89 means that 89% of the variance in the crop yield is explained by the model. This is a strong indicator that your model fits the data well.
#CONCLUSION:HIGH MSE VALUE OF 0.51 SUGGESTS THE SQUARED DEVIATION OF THIS MODEL AND ACTUAL CROP YEILDS,R-SQUARED VALUE OF 0.89 SUGGESTS A GREAT FITTING AND AN OVERALL VARIANCE, INDICATING A GREAT PERFORMANCE.